Just importing stuff

In [2]:
%pip install keras
%pip install PyQt5
import pandas as pd
import numpy as np
import tensorflow as tf
import PyQt5
import matplotlib.pyplot as plt
import datetime as dt



Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'e:\Python\python3.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'e:\Python\python3.exe -m pip install --upgrade pip' command.


In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,LSTM
from tensorflow.keras.optimizers import Adam

In [4]:
data = pd.read_csv("../complete_merged.csv",index_col=0)

Consecutive date checking function (Some districts had partial data)

In [32]:
#Testing datetime consecutiveness


def checkConsecutiveDates(date_strs):
    dates = [dt.datetime.strptime(d, "%Y-%m-%d") for d in date_strs]

    date_ints = set([d.toordinal() for d in dates])
    if len(date_ints) == 1:
        return True
    elif max(date_ints) - min(date_ints) == len(date_ints) - 1:
        return True
    else:
        return False
    


def split_sequence(date_sequence,temp_sequence, n_steps,n_features):
    X, y = list(), list()
    for i in range(len(temp_sequence)):
    # find the end of this pattern
        end_ix = i + n_steps
 # check if we are beyond the sequence
        if end_ix > len(temp_sequence)-1:
            break
 # gather input and output parts of the pattern
        seq_x, seq_y = temp_sequence[i:end_ix], temp_sequence[end_ix]
        date_seq = date_sequence[i:end_ix]
        # print(checkConsecutive(seq_x))
        if(checkConsecutiveDates(date_seq)):
            X.append(seq_x)
            y.append(seq_y)
    X,y = np.array(X), np.array(y)
    X = X.reshape((X.shape[0], X.shape[1], n_features))
        
    return X,y

def get_data(district,column):
    return data[data['district'] == district].sort_values(axis = 0,by = "date")[column].to_list()
   



# date_strs = ['02-29-2012', '02-28-2012', '03-01-2012']
date_strs = ['1999-12-31','2000-01-01']
print(checkConsecutiveDates(date_strs))

X,y = split_sequence(['2021-01-09','2021-01-11','2021-01-12','2021-01-13','2021-01-14'],[10, 20, 30, 40, 50],3,1)
for i in range(len(X)):
    print(X[i], y[i])





True
[[20]
 [30]
 [40]] 50


Split Sequence(Standard function for splitting data for univariate lstm)

Function to evaluate model(Mean Square Error)

In [13]:
def evalLSTM(model,X,y,n_steps,n_features):
    y_predict = list()
    for i in range(X.shape[0]):
        x_input = X[i]
        x_input = x_input.reshape((1, n_steps, n_features))
        predicted_value = model.predict(x_input, verbose=0)[0][0]
        print(len(y_predict),"of",X.shape[0])
        y_predict.append(predicted_value)
    
    mse = tf.keras.losses.MeanSquaredError()
    
    return {'mse':mse(y_predict,y).numpy(),'y_predict':y_predict}


Defining the model

In [14]:
#different optimizers and loss functions need to be tested yet
n_steps = 10
n_features = 1
epochs =20
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')



Basic check for Adilabad

In [33]:
data_Adilabad_temp = get_data('Adilabad','temp_max')
data_Adilabad_date = get_data('Adilabad','date')
# print(data_Adilabad_date)
X,y = split_sequence(data_Adilabad_date,data_Adilabad_temp,n_steps,n_features)

model.fit(X, y, epochs=200, verbose=4)
mse = evalLSTM(model,X,y,10,1)

mse
#3.36 for 200 epochs


Epoch 1/200
Epoch 2/200
Epoch 3/200
Epoch 4/200
Epoch 5/200
Epoch 6/200
Epoch 7/200
Epoch 8/200
Epoch 9/200
Epoch 10/200
Epoch 11/200
Epoch 12/200
Epoch 13/200
Epoch 14/200
Epoch 15/200
Epoch 16/200
Epoch 17/200
Epoch 18/200
Epoch 19/200
Epoch 20/200
Epoch 21/200
Epoch 22/200
Epoch 23/200
Epoch 24/200
Epoch 25/200
Epoch 26/200
Epoch 27/200
Epoch 28/200
Epoch 29/200
Epoch 30/200
Epoch 31/200
Epoch 32/200
Epoch 33/200
Epoch 34/200
Epoch 35/200
Epoch 36/200
Epoch 37/200
Epoch 38/200
Epoch 39/200
Epoch 40/200
Epoch 41/200
Epoch 42/200
Epoch 43/200
Epoch 44/200
Epoch 45/200
Epoch 46/200
Epoch 47/200
Epoch 48/200
Epoch 49/200
Epoch 50/200
Epoch 51/200
Epoch 52/200
Epoch 53/200
Epoch 54/200
Epoch 55/200
Epoch 56/200
Epoch 57/200
Epoch 58/200
Epoch 59/200
Epoch 60/200
Epoch 61/200
Epoch 62/200
Epoch 63/200
Epoch 64/200
Epoch 65/200
Epoch 66/200
Epoch 67/200
Epoch 68/200
Epoch 69/200
Epoch 70/200
Epoch 71/200
Epoch 72/200
Epoch 73/200
Epoch 74/200
Epoch 75/200
Epoch 76/200
Epoch 77/200
Epoch 78

{'mse': 3.3073234410859675,
 'y_predict': [32.90521,
  29.99476,
  31.974672,
  32.406506,
  33.111984,
  33.442135,
  32.281944,
  31.842768,
  32.069874,
  31.200079,
  30.902596,
  30.910368,
  30.980927,
  31.700478,
  31.540161,
  30.50757,
  29.986095,
  30.613216,
  31.103876,
  31.996086,
  31.53308,
  31.868393,
  30.570007,
  33.48458,
  36.965576,
  37.646717,
  39.922215,
  34.19121,
  33.95171,
  32.93657,
  33.409065,
  34.89167,
  30.597439,
  31.826527,
  31.745985,
  30.664267,
  29.507141,
  31.144083,
  30.933887,
  33.07612,
  34.33256,
  34.14383,
  33.37293,
  32.779655,
  34.71787,
  34.44299,
  34.119823,
  34.871216,
  35.31547,
  35.76566,
  32.556416,
  34.104424,
  37.43602,
  37.181423,
  39.11407,
  34.079056,
  32.63869,
  31.27921,
  32.75204,
  33.95296,
  30.472048,
  30.726208,
  38.596783,
  37.181404,
  37.515533,
  32.727478,
  30.120193,
  34.448814,
  35.290707,
  36.704067,
  36.44018,
  34.92218,
  37.013523,
  37.656536,
  37.596027,
  36.6831

In [39]:

date_X,date_y = split_sequence(data_Adilabad_date,data_Adilabad_date,n_steps,n_features)

%matplotlib qt
date_y = np.asarray(date_y, dtype='datetime64[s]')

plt.plot(date_y,y)
plt.plot(date_y,mse['y_predict'])
plt.show()


In [18]:
mse

{'mse': 3.758542518068985,
 'y_predict': [34.063923,
  30.74429,
  31.821268,
  33.09446,
  32.978916,
  33.76412,
  33.39025,
  32.60031,
  32.898975,
  31.907433,
  31.671135,
  31.62411,
  31.609207,
  32.207317,
  32.027023,
  31.270308,
  30.753977,
  31.229898,
  31.625242,
  32.412327,
  32.110947,
  32.469135,
  31.400623,
  33.749744,
  36.602192,
  37.64719,
  40.486607,
  37.085938,
  35.00787,
  34.764706,
  34.358955,
  34.793823,
  31.198313,
  31.500618,
  32.09528,
  31.784075,
  30.811777,
  31.423,
  31.44716,
  33.25067,
  34.585846,
  34.494305,
  34.046806,
  33.639538,
  35.299465,
  35.205395,
  34.873165,
  35.56994,
  35.90821,
  36.26901,
  33.19153,
  34.760944,
  37.412724,
  37.779724,
  41.72584,
  35.585945,
  33.96252,
  33.616425,
  33.95037,
  33.450523,
  31.414082,
  31.413187,
  37.71043,
  37.46911,
  37.48908,
  35.652008,
  31.023726,
  34.111565,
  36.156715,
  37.437424,
  36.74993,
  36.353226,
  36.87419,
  37.802937,
  37.83393,
  37.64048,


Various window sizes were used on Adilabad for optimization.Results were as follows:

[10-3.289,
 20-55142.400,
 30-6766769.447,
 40-216242835.855,
 50-14841958357.522,
 60-793333303283.484]


[7-116.768,
 8-28.928,
 9-5.358,
 10-3.289,
 11-5.317]

Concluding that 10 is an optimal window size

Training on complete data of all districts

In [ ]:
model = tf.keras.models.load_model('saved_model/model_e20_w5')
n_steps = 5
n_features = 1


'mse': 3.042855618505585 when trained on all data and tested against random 20 epochs

In [ ]:
data_unique_district_temp = get_data('Adilabad','temp_max')
data_unique_district_date = get_data('Adilabad','date')
    
  
X,y = convertUnivariateData(data_unique_district_date, data_unique_district_temp,n_steps,n_features)
metrics = evalLSTM(model,X,y,5,1)


In [ ]:
temp_2023 = np.array([29.0,28.0,27.0,21.0,19.0,26.0,27.0,28.0,30.0,31.0])
temp_window = temp_2023[-10:]
for i in range(100):
    temp_window = temp_window.reshape(1, n_steps, n_features)
    predicted_value = model.predict(temp_window, verbose=0)[0][0]
    print(predicted_value)
    temp_2023 = np.append(temp_2023, predicted_value)
    temp_window = temp_2023[-10:]


Multiple window size training


In [37]:

%%script false


# steps = [5,10,20,30,60,90]
steps = [90]
for i in steps:
    n_steps = i
    n_features = 1
    epochs =20
    model = Sequential()
    model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')

    i = 1
    for district in data['district'].unique():
        print(str(i) + "out of "+str(len(data['district'].unique())))
        
        data_unique_district_temp = get_data(district,'temp_max')
        data_unique_district_date = get_data(district,'date')
        
        print(district,str(len(data_unique_district_temp)))
        X,y = split_sequence(data_unique_district_date, data_unique_district_temp,n_steps,n_features)
        model.fit(X, y, epochs=epochs, verbose=0)
        i = i+1

    model.save('saved_model/model_e20_w'+str(n_steps))

Couldn't find program: 'false'


In [42]:
# %%script false

final_df = pd.DataFrame()



i = 1
# for district in data['district'].unique():
for district in ['Adilabad']:
    temp_df = pd.DataFrame()
    print(str(i) + "out of "+str(len(data['district'].unique())))
    
    data_unique_district_temp = get_data(district,'temp_max')
    data_unique_district_date = get_data(district,'date')
   
    print(district,str(len(data_unique_district_temp)))
    X,y = split_sequence(data_unique_district_date, data_unique_district_temp,n_steps,n_features)
    date_X,date_y = split_sequence(data_unique_district_date,data_unique_district_date,n_steps,n_features)
    metrics = evalLSTM(model,X,y,10,1)

    temp_df['date'] = date_y
    temp_df['predict_temp_max'] = metrics['y_predict']
    temp_df['district'] = district
    temp_df['window_size'] = 10

    final_df.concat(temp_df)
    
    
    i = i+1

temp_df

1out of 46
Adilabad 1733
0 of 1714
1 of 1714
2 of 1714
3 of 1714
4 of 1714
5 of 1714
6 of 1714
7 of 1714
8 of 1714
9 of 1714
10 of 1714
11 of 1714
12 of 1714
13 of 1714
14 of 1714
15 of 1714
16 of 1714
17 of 1714
18 of 1714
19 of 1714
20 of 1714
21 of 1714
22 of 1714
23 of 1714
24 of 1714
25 of 1714
26 of 1714
27 of 1714
28 of 1714
29 of 1714
30 of 1714
31 of 1714
32 of 1714
33 of 1714
34 of 1714
35 of 1714
36 of 1714
37 of 1714
38 of 1714
39 of 1714
40 of 1714
41 of 1714
42 of 1714
43 of 1714
44 of 1714
45 of 1714
46 of 1714
47 of 1714
48 of 1714
49 of 1714
50 of 1714
51 of 1714
52 of 1714
53 of 1714
54 of 1714
55 of 1714
56 of 1714
57 of 1714
58 of 1714
59 of 1714
60 of 1714
61 of 1714
62 of 1714
63 of 1714
64 of 1714
65 of 1714
66 of 1714
67 of 1714
68 of 1714
69 of 1714
70 of 1714
71 of 1714
72 of 1714
73 of 1714
74 of 1714
75 of 1714
76 of 1714
77 of 1714
78 of 1714
79 of 1714
80 of 1714
81 of 1714
82 of 1714
83 of 1714
84 of 1714
85 of 1714
86 of 1714
87 of 1714
88 of 1714
89 of 

,Adilabad,date
0,32.905209,2018-01-11
1,29.994761,2018-01-12
2,31.974672,2018-01-13
3,32.406506,2018-01-14
4,33.111984,2018-01-15
...,...,...
1709,32.657871,2022-09-26
1710,33.213398,2022-09-27
1711,32.543259,2022-09-28
1712,33.276501,2022-09-29
